In [70]:
import requests 
import json
from bs4 import BeautifulSoup

In [81]:
base = "https://academicjournals.org"
num = 0 # global index for creating a name

In [72]:
def create_json(title, authors, key_words, text):
    """Creating a json file with fields"""
    data = {
        'title': title,
        'authors': authors,
        'text': text,
        'key_words': key_words
    }
    with open(f'AJBM parse/{num}.json', 'w') as f:
        json.dump(data, f)

In [73]:
def take_auth(soup):
    '''Parse the soup to take an author'''
    authors = []
    for user in soup.find_all("div", "user"):
        authors.append(user.find("h5").text)
    return authors

In [80]:
def parse_links_pack(links):
    '''Create a json file for each page in links'''
    for link in links:
        global num
        num += 1 # increment an index-name
        req = requests.get(link)
        sub_soup = BeautifulSoup(req.text, "html.parser")
        title = sub_soup.find("h2").text
        authors = take_auth(sub_soup)
        all_p = sub_soup.find("div", "articles abstract_details").find_all("p")
        #in some cases this part throw an exeption but all pages have a standart format, idk. Anyway
        try:
            all_p[-1].strong.decompose()
            key_words = all_p[-1].text.strip()[:-1].split(", ")
            text = all_p[1].text.strip()
        except:
            key_words = []
            text = ""
        create_json(title, authors, key_words, text)

In [75]:
def take_links_from_page(soup):
    '''every soup has about 50 links to needed pages in it'''
    links = []
    for a in soup.find_all("a", "btn btn-success"):
        href = a.get("href")
        links.append(base+href)
    return links

In [ ]:
# circle for creating a soup from each base-search url
for i in range(1, 81):
    url = f"https://academicjournals.org/articles/j_articles/AJBM/page:{i}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    links = take_links_from_page(soup)
    parse_links_pack(links)

In [ ]:
# comands below were a little lab while parsing. It stay here in case it would be needed

In [15]:
link = "https://academicjournals.org/journal/AJBM/article-abstract/4D3FE8370687"

In [16]:
req = requests.get(link)
sub_soup = BeautifulSoup(req.text, "html.parser")

In [32]:
sub_soup.find("div", "articles abstract_details").find_all("p")[-1].text.split(": ")[1].split(", ")[-1][:-1]

'Uganda'

In [60]:
sub_soup.find("div", "articles abstract_details").find_all("p")[-1].strong.decompose()

In [58]:
links[19]

'https://academicjournals.org/journal/AJBM/article-abstract/0330EFA69299'

In [65]:
sub_soup.find("div", "articles abstract_details").find_all("p")[-1].text.strip()[:-1].split()

['Stakeholders,',
 'resource',
 'mobilization,',
 'sustainability,',
 'and',
 'donors',
 'funded',
 'potato',
 'projects,',
 'Kabale',
 'District,',
 'Uganda']

In [ ]:
lk = "https://academicjournals.org/journal/AJBM/article-abstract/D8AB70E53472"